In [24]:
import pandas as pd
import requests
import os
import time

def area_list(Area, headers):
    area_json = 'https://static.104.com.tw/category-tool/json/Area.json'
    response = requests.get(url=area_json, headers=headers)
    df_area = pd.DataFrame(response.json()[0]['n'])
    df_area = df_area.explode('n', ignore_index=True)
    df_area['des2'] = df_area['n'].apply(lambda x: x['des'])
    df_area['no2'] = df_area['n'].apply(lambda x: x['no'])
    df_area = df_area.loc[:, ['des', 'no', 'des2', 'no2']]
    Area_list = list()
    for area in Area:
        for index in df_area.index:
            if df_area.iloc[index]['des']  == area:
                areacode = df_area.iloc[index]['no']
            elif df_area.iloc[index]['des2']  == area:
                areacode = df_area.iloc[index]['no2']
        Area_list.append(areacode)
    return Area_list


start_time = time.time()
search_list = {
    'jobName': '職務名稱',
    'custName': '公司名稱',
    'jobAddrNoDesc': '工作地點',
    'jobAddress': '詳細地址',
    'optionEdu': '學歷',
    'periodDesc': '工作經歷',
    'salaryDesc': '待遇',
    'applyDesc': '應徵人數',
    'description': '工作內容',
    'coIndustryDesc': '產業描述',
    'appearDate': '發布日期',
    'link': '網頁連結'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
    'Referer': 'https://www.104.com.tw/jobs/main/'
}
folder_path = './104'
max_page = 10        # 使用者可自行輸入
ro = '1'  # 0: 全部, 1: 全職, 2: 兼職
isnew = '7'  # 幾日內更新的工作
keyword = '軟體工程師'
Area = ['台北市中正區', '新北市']  # 想搜尋的地區
Area = area_list(Area, headers)
df = list()
for area in Area:
    for page in range(1, max_page):
        url = f'https://www.104.com.tw/jobs/search/list?ro={ro}&isnew={isnew}&kwop=7&keyword={keyword}&area={area}&order=15&asc=0&page={page}&mode=l'
        response = requests.get(url=url, headers=headers)
        ndf = pd.DataFrame(response.json()['data']['list'])
        df.append(ndf)
        if ndf.shape[0] < 30:
            break
df = pd.concat(df, ignore_index=True)
df = df.rename(columns=search_list)  # 可寫成字典更改column名或index名
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
df.to_csv(os.path.join(folder_path, f'{keyword}.csv'), index=0, encoding='utf-8-sig')
end_time = time.time()
print('運算時間:\t', end_time - start_time)
df

運算時間:	 31.748793601989746


,jobType,jobNo,職務名稱,jobNameSnippet,jobRole,jobRo,jobAddrNo,工作地點,詳細地址,工作內容,...,tags,landmark,網頁連結,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,1,8299084,CRD 手機遊戲工程師【歡迎應屆畢業生】,CRD 手機遊戲工程師【歡迎應屆畢業生】,1,1,6001001001,台北市中正區,羅斯福路二段100號10樓,1. 使用Cocos開發遊戲(無經驗可)\n2. 使用JavaScript語言\n3. 維護...,...,[員工200人],距捷運古亭站50公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,CRD 手機遊戲工程師【歡迎應屆畢業生】,慧邦科技股份有限公司,121.5223326,25.0267506,0,"[資訊工程相關, 數學及電算機科學學科類, 電機電子工程相關]"
1,1,11867437,RD 網頁/PHP工程師【歡迎應屆畢業生】,RD 網頁/PHP工程師【歡迎應屆畢業生】,1,1,6001001001,台北市中正區,羅斯福路二段100號10樓,開發web/手機遊戲\n－ 遊戲功能開發\n－ 活動開發及官網維護\n－ 資料庫操作及數據處...,...,[員工200人],距捷運古亭站50公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,RD 網頁/PHP工程師【歡迎應屆畢業生】,慧邦科技股份有限公司,121.5223326,25.0267506,0,"[資訊工程相關, 資訊管理相關, 其他數學及電算機科學相關]"
2,1,13034928,C程式設計人員,C程式設計人員,1,1,6001001001,台北市中正區,或新北市板橋區,※必備資格條件：\r\n教育部認可之大學(含)以上資訊相關系所畢業。\r\n※具下列資格條件...,...,[員工290人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,C程式設計人員,財團法人中華民國證券櫃檯買賣中心,121.5198716,25.0421407,0,[資訊管理相關]
3,1,13034934,C#程式設計人員,C#程式設計人員,1,1,6001001001,台北市中正區,或新北市板橋區,※必備資格條件：\r\n教育部認可之大學(含)以上資訊相關系所畢業。\r\n※具下列資格條件...,...,[員工290人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,C#程式設計人員,財團法人中華民國證券櫃檯買賣中心,121.5198716,25.0421407,0,[資訊管理相關]
4,0,12376125,軟體工程師 Software Engineer- Embedded System,<em class='b-txt--highlight'>軟體工程師</em> <em cl...,1,1,6001001001,台北市中正區,杭州南路一段15-1號10樓之一,【Job Description】 \n1. Develop Low latency Tr...,...,[員工60人],距捷運善導寺站410公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,軟體工程師 Software Engineer- Embedded System,威旭資訊有限公司,121.5267675,25.0427628,0,"[電機電子工程相關, 資訊工程相關]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,0,13051700,智慧能源前端工程師,智慧能源前端<em class='b-txt--highlight'>工程師</em>,1,1,6001002025,新北市五股區,,職務類別：[[[軟體設計工程師]]]、Internet程式設計師、通訊[[[軟體工程師]]]...,...,"[上市上櫃, 員工11000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_d_relevance,智慧能源前端工程師,台塑_臺灣塑膠工業股份有限公司,121.4386586,25.0848317,0,[電機電子工程相關]
544,0,12911536,【S】Linux / Yocto / OpenWRT BSP 軟體工程師 (土城),【S】Linux / Yocto / OpenWRT BSP <em class='b-tx...,1,1,6001002016,新北市土城區,民生街4號,1. OpenWRT/ Yocto BSP 開發環境、功能整合\n2. OpenWRT/ L...,...,[],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_d_relevance,【S】Linux / Yocto / OpenWRT BSP 軟體工程師 (土城),富智康國際股份有限公司(鴻海集團),121.4143447,24.9613226,0,"[資訊工程相關, 資訊管理相關, 電機電子工程相關]"
545,2,12908531,資深.NET 軟體開發工程師 (新店),資深.NET <em class='b-txt--highlight'>軟體開發工程師</e...,1,1,6001002011,新北市新店區,復興路43號10樓,1.依產品及專案需求，進行 ASP.NET MVC 應用程式開發。\n2.依產品及專案需求，...,...,[員工40人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_d_relevance,資深.NET 軟體開發工程師 (新店),睿傳數據股份有限公司,121.5369557,24.9866876,0,[資訊工程相關]
546,0,7941327,自動控制軟體工程師(C# C++),自動控制<em class='b-txt--highlight'>軟體工程師</em>(C#...,1,1,6001002018,新北市樹林區,八德路432號,1.參與應用[[[軟體]]]之規劃與設計\n2.參與自動化機台[[[軟體]]]架構與系統設計...,...,[員工45人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_d_relevance,自動控制軟體工程師(C# C++),翔慶精密工業有限公司,121.4017587,24.9704558,0,"[數學及電算機科學學科類, 工程學科類]"


In [22]:
import pandas as pd
import requests
def area_list(Area, headers):
    area_json = 'https://static.104.com.tw/category-tool/json/Area.json'
    response = requests.get(url=area_json, headers=headers)
    df_area = pd.DataFrame(response.json()[0]['n'])
    df_area = df_area.explode('n', ignore_index=True)
    df_area['des2'] = df_area['n'].apply(lambda x: x['des'])
    df_area['no2'] = df_area['n'].apply(lambda x: x['no'])
    df_area = df_area.loc[:, ['des', 'no', 'des2', 'no2']]
    Area_list = list()
    for area in Area:
        for index in df_area.index:
            if df_area.iloc[index]['des']  == area:
                areacode = df_area.iloc[index]['no']
            elif df_area.iloc[index]['des2']  == area:
                areacode = df_area.iloc[index]['no2']
        Area_list.append(areacode)
    return Area_list

In [23]:
import requests
area_json = 'https://static.104.com.tw/category-tool/json/Area.json'
ro = '1'  # 0: 全部, 1: 全職, 2: 兼職
isnew = '7'  # 幾日內更新的工作
keyword = '軟體工程師'
area = '6001001001'  # 台北市
url = f'https://www.104.com.tw/jobs/search/list?ro={ro}&isnew={isnew}&kwop=7&keyword={keyword}&area={area}&order=15&asc=0&page=2&mode=l'
response = requests.get(url=url, headers=headers)
response.json()

{'status': 200,
 'action': [],
 'data': {'query': {'ro': 1,
   'jobcat': '',
   'isnew': 7,
   'kwop': 7,
   'keyword': '軟體工程師',
   'expansionType': 'area,spec,com,job,wf,wktm',
   'excludeKeyword': '',
   'area': '6001001001',
   'indcat': '',
   'expcate': '',
   'edu': '',
   'order': 15,
   'asc': 0,
   'zone': '',
   'dep': '',
   'dis_role': '',
   'lang': '',
   'sr': '',
   'sctp': '',
   'scmin': '',
   'scmax': '',
   'scstrict': '',
   'scneg': '',
   'excludeJobKeyword': '',
   'excludeCompanyKeyword': '',
   'excludeCompanyByCustno': '',
   'excludeIndustryCat': '',
   's9': '',
   's5': '',
   'wktm': '',
   'startby': '',
   'rostatus': '',
   'page': 2,
   'wf': '',
   'jobexp': '',
   'wt': '',
   'mode': 'l',
   'jobsource': '',
   'utm_source': '',
   'utm_medium': '',
   'utm_campaign': '',
   'c': '',
   'custNo': '',
   'jobNo': '',
   'dist': '',
   'lat': '',
   'lon': '',
   'mrt': '',
   'indArea': '',
   'newZone': '',
   'searchTempExclude': '',
   'manage':